In [1]:
import sys
print(sys.executable)

/home/maxime/dev/python/sndx/.venv/bin/python3.12


In [27]:
import subprocess
import time
import asyncio
import secrets
from pyppeteer import launch

In [48]:
class Session:
    def __init__(self, id: str):
        self.id = id
        self.sink_name = f"sndx-{id}"
        self.profile_path = f"/tmp/sndx-profile-{id}"

    def __repr__(self):
        return f"Session(id={self.id!r})"
        

def generate_session():
    return Session(secrets.token_urlsafe(4)[:6])

In [49]:
class Sink:
    def __init__(self, name: str, module_id: str):
        self.name = name
        self.module_id = module_id

    def __repr__(self):
        return f"Sink(name={self.name!r}, module_id={self.module_id!r})"


def create_sink(session):
    module_id = subprocess.check_output(
        ["pactl", "load-module", "module-null-sink", session.sink_name],
        text=True).strip()
    return Sink(session.sink_name, module_id)


def remove_sink(sink):
    subprocess.Popen([
        "pactl", "unload-module", sink.module_id,
    ])

In [45]:
session = generate_session()
print(session)
sink = create_sink(session)
print(sink)
remove_sink(sink)

Session(id='ddPjEA')
Sink(name='sndx-ddPjEA', module_id='536870916')


In [50]:
session = generate_session()

In [53]:
browser = await launch({
    "headless": False,
    #"userDataDir": session.profile_path,
    #"args": [
    #    f"--audio-output-sink={session.sink_name}",
    #],
})

BrowserError: Browser closed unexpectedly:


In [ ]:
page = await browser.newPage()